In [1]:
import numpy as np
import pandas as pd
 
import matplotlib.pyplot as plt

import json

import re

import pickle

import nltk
from nltk.corpus import stopwords

from pymystem3 import Mystem

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, GlobalAveragePooling1D, Flatten, BatchNormalization
from tensorflow.keras import utils, Input, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/artemiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artemiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# data collected on June 1, 2024 at 4:00 AM
DATASET_PATH = './data/data.json'

In [4]:
with open(DATASET_PATH, 'r') as file:
    data = json.load(file)

max_length = max([len(data[elem]) for elem in data])

for elem in data:
    while len(data[elem]) < max_length:
        data[elem].append('')
        
max_length

1500

In [5]:
data = json.dumps(data)

In [6]:
df = pd.read_json(data)

df

/var/folders/m1/7thbrsg96m1_wm0p48jm8d7c0000gn/T/ipykernel_74404/792015447.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data)


,АВТО БАТЯ,АЭРОФЛОТ,Телеграмма РЖД
0,Зачем переплачивать за доставку\n\n[АВТО БАТЯ]...,,
1,«Сравни.ру» огласил победителя ежегодной преми...,,
2,В Нигерии толпа собралась посмотреть на горящу...,,
3,"**Фургон задавил челябинку, которая отвлеклась...",,
4,[МФЦ](https://t.me/+wk0wdjBDPJJmMWMy) пришел в...,**Аэрофлот — партнёр шоу «Восхождение» Фонда С...,**Интегрировали станцию Кутузовская МЦД-4 с ме...
...,...,...,...
1495,**Десятилетний мальчик скатился с горки прямо ...,**В гостях у Аэрофлота**\n\n👧🧒🏻Накануне Дня за...,"🚆 **Перевели «на цифру» ст. Апрелевка, конечну..."
1496,Жуткая ситуация на дороге\n\n[АВТО БАТЯ](https...,**✈️ В Сочи из Самары!\n**\nЗапускаем с 1 июня...,**Погрузка экспортных грузов **[**в порты**](h...
1497,Волшебный бардачок\n\n[АВТО БАТЯ](https://t.me...,**✈️ Июньский номер журнала «Аэрофлот» - уже н...,👨‍🦽👩‍🦯 **Вокзалы для каждого: создаём инклюзив...
1498,"**«Давай эту тоже вперёд толкнём, пацаны»**\n\...",**✈️🌍 Как путешествовать по миру сегодня? \n\...,


In [7]:
df = df.melt(var_name='channel_name', value_name='post_text')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   channel_name  4500 non-null   object
 1   post_text     4500 non-null   object
dtypes: object(2)
memory usage: 70.4+ KB


In [9]:
NUM_WORDS = 5_000                                 # Максимальное количество слов
MAX_NEWS_LEN = 100                                # Максимальная длина новости
NB_CLASSES = df['channel_name'].unique().shape[0] # Количество классов

In [10]:
def remove_special_characters(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [11]:
df.replace("", pd.NA, inplace=True)
df.dropna(inplace=True)

# Преобразование текста в нижний регистр
df['post_text'] = df['post_text'].apply(lambda x: x.lower())

# Удаление специальных символов и цифр
df['post_text'] = df['post_text'].apply(remove_special_characters)

# Токенизация текста
df['tokens'] = df['post_text'].apply(nltk.word_tokenize)

# Удаление стоп-слов
stop_words = set(stopwords.words('russian'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Лемматизация текста
# Не забудьте установить пакеты для лемматизации, например pymystem3

m = Mystem()
df['tokens'] = df['tokens'].apply(lambda x: [m.lemmatize(word)[0] for word in x])

# Преобразование токенов обратно в текст
df['processed_text'] = df['tokens'].apply(lambda x: ' '.join(x))
# Вы можете добавить другие шаги preprocessing текста в зависимости от ваших потребностей

# В итоговом датафрейме будет колонка 'processed_text' с обработанным текстом

In [12]:
df

,channel_name,post_text,tokens,processed_text
0,АВТО БАТЯ,зачем переплачивать за доставку авто батя http...,"[переплачивать, доставка, авто, батя, https, m...",переплачивать доставка авто батя https me join...
1,АВТО БАТЯ,сравни ру огласил победителя ежегодной премии...,"[сравнивать, ру, оглашать, победитель, ежегодн...",сравнивать ру оглашать победитель ежегодный пр...
2,АВТО БАТЯ,в нигерии толпа собралась посмотреть на горящу...,"[нигерия, толпа, собираться, посмотреть, горет...",нигерия толпа собираться посмотреть гореть зап...
3,АВТО БАТЯ,фургон задавил челябинку которая отвлеклась н...,"[фургон, задавливать, челябинка, который, отвл...",фургон задавливать челябинка который отвлекать...
4,АВТО БАТЯ,мфц https me wk wdjbdpjjmmwmy пришел в telegr...,"[мфц, https, me, wk, wdjbdpjjmmwmy, приходить,...",мфц https me wk wdjbdpjjmmwmy приходить telegr...
...,...,...,...,...
4493,Телеграмма РЖД,лучшие из лучших определятся в честной борьбе...,"[хороший, хороший, определяться, честный, борь...",хороший хороший определяться честный борьба ек...
4494,Телеграмма РЖД,с начала года обновили электрички в регионах ...,"[начинать, год, обновлять, электричка, регион,...",начинать год обновлять электричка регион конец...
4495,Телеграмма РЖД,перевели на цифру ст апрелевка конечную остан...,"[переводить, цифра, ст, апрелевка, конечный, о...",переводить цифра ст апрелевка конечный останов...
4496,Телеграмма РЖД,погрузка экспортных грузов в порты https comp...,"[погрузка, экспортный, груз, порт, https, comp...",погрузка экспортный груз порт https company rz...


In [13]:
df_encoded = pd.get_dummies(df['channel_name'], dtype=int)

# Объединение закодированных колонок с исходным датафреймом
df = pd.concat([df, df_encoded], axis=1)

# Удаление исходной колонки 'channel_name'
df.drop('channel_name', axis=1, inplace=True)

df

,post_text,tokens,processed_text,АВТО БАТЯ,АЭРОФЛОТ,Телеграмма РЖД
0,зачем переплачивать за доставку авто батя http...,"[переплачивать, доставка, авто, батя, https, m...",переплачивать доставка авто батя https me join...,1,0,0
1,сравни ру огласил победителя ежегодной премии...,"[сравнивать, ру, оглашать, победитель, ежегодн...",сравнивать ру оглашать победитель ежегодный пр...,1,0,0
2,в нигерии толпа собралась посмотреть на горящу...,"[нигерия, толпа, собираться, посмотреть, горет...",нигерия толпа собираться посмотреть гореть зап...,1,0,0
3,фургон задавил челябинку которая отвлеклась н...,"[фургон, задавливать, челябинка, который, отвл...",фургон задавливать челябинка который отвлекать...,1,0,0
4,мфц https me wk wdjbdpjjmmwmy пришел в telegr...,"[мфц, https, me, wk, wdjbdpjjmmwmy, приходить,...",мфц https me wk wdjbdpjjmmwmy приходить telegr...,1,0,0
...,...,...,...,...,...,...
4493,лучшие из лучших определятся в честной борьбе...,"[хороший, хороший, определяться, честный, борь...",хороший хороший определяться честный борьба ек...,0,0,1
4494,с начала года обновили электрички в регионах ...,"[начинать, год, обновлять, электричка, регион,...",начинать год обновлять электричка регион конец...,0,0,1
4495,перевели на цифру ст апрелевка конечную остан...,"[переводить, цифра, ст, апрелевка, конечный, о...",переводить цифра ст апрелевка конечный останов...,0,0,1
4496,погрузка экспортных грузов в порты https comp...,"[погрузка, экспортный, груз, порт, https, comp...",погрузка экспортный груз порт https company rz...,0,0,1


In [14]:
# shuffled_df = df.sample(frac=1).reset_index(drop=True)

# df = shuffled_df

In [15]:
tokenizer = Tokenizer(num_words=NUM_WORDS)

In [16]:
tokenizer.fit_on_texts(df['processed_text'])

In [17]:
tokenizer.word_index

{'ru': 1,
 'https': 2,
 'me': 3,
 'год': 4,
 'авто': 5,
 'батя': 6,
 'm': 7,
 'joinchat': 8,
 'vvd': 9,
 'xytni': 10,
 'москва': 11,
 'aeroflot': 12,
 'аэрофлот': 13,
 'поезд': 14,
 'www': 15,
 'который': 16,
 'ржд': 17,
 'пассажир': 18,
 'рейс': 19,
 'новый': 20,
 'наш': 21,
 'это': 22,
 'билет': 23,
 'млн': 24,
 'россия': 25,
 'тонна': 26,
 'время': 27,
 'utm': 28,
 'водитель': 29,
 'дорога': 30,
 'также': 31,
 'день': 32,
 'работа': 33,
 'место': 34,
 'сайт': 35,
 'путь': 36,
 'перевозка': 37,
 'тыс': 38,
 'первый': 39,
 'станция': 40,
 'движение': 41,
 'направление': 42,
 'полет': 43,
 'петербург': 44,
 'быть': 45,
 'область': 46,
 'автомобиль': 47,
 'путешествие': 48,
 'app': 49,
 'самый': 50,
 'один': 51,
 'search': 52,
 'санкт': 53,
 'становиться': 54,
 'человек': 55,
 'sb': 56,
 'вагон': 57,
 'cabin': 58,
 'adults': 59,
 'children': 60,
 'infants': 61,
 'routes': 62,
 'железный': 63,
 'economy': 64,
 'летний': 65,
 'свой': 66,
 'su': 67,
 'сегодня': 68,
 'вокзал': 69,
 'железно

In [18]:
TOKENIZER_PATH = './learned_models'

with open(f'{TOKENIZER_PATH}/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
sequences = tokenizer.texts_to_sequences(df['processed_text'])

In [20]:
EXAMPLE_INDEX = 1

print(df.iloc[EXAMPLE_INDEX]['processed_text'])
print(sequences[EXAMPLE_INDEX])

сравнивать ру оглашать победитель ежегодный премия номинация каско год исход весьма закономерный пользователь маркетплейс важно эксперт отдавать https www vedomosti ru press prosto bit liderom ingosstrah pobedil nominatsii kasko goda ezhegodnoi premii sravni предпочтение страховой гигант ингосстрах высоко оценивать предлагать клиент арсенал продукт добавлять кредит доверие полюбиться обладатель полис компания мобильный приложение ingomobile благодаря который услуга страховщик доступный буквально движение палец премия сравнивать молодой проводиться второй учитывать репутация сам сравнивать ру обладание приз считаться весьма значимый страховой компания
[4020, 2546, 595, 1986, 850, 941, 4, 4556, 1173, 4021, 1174, 1681, 1545, 2, 15, 1, 1605, 4022, 4557, 3568, 1682, 1175, 345, 467, 1207, 681, 2972, 4558, 1786, 74, 202, 160, 487, 16, 171, 197, 1683, 41, 4023, 850, 4020, 596, 1176, 153, 1606, 488, 4020, 2546, 851, 2547, 4556, 2108, 3568, 74]


In [21]:
x_train = pad_sequences(sequences, maxlen=MAX_NEWS_LEN)

x_train

array([[   0,    0,    0, ...,    7,    9,   10],
       [   0,    0,    0, ..., 2108, 3568,   74],
       [   0,    0,    0, ...,    7,    9,   10],
       ...,
       [   0,    0,    0, ...,   54,  515,  344],
       [   0,    0,    0, ...,  439,   24,   26],
       [   0,    0,    0, ...,   69,   69, 1602]], dtype=int32)

In [22]:
y_train = df.drop(['processed_text', 'post_text', 'tokens'], axis=1)

y_train 

,АВТО БАТЯ,АЭРОФЛОТ,Телеграмма РЖД
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
4493,0,0,1
4494,0,0,1
4495,0,0,1
4496,0,0,1


In [23]:
DIR_DATA_PATH = './data'

with open(f'{DIR_DATA_PATH}/x_train.json', 'w') as f:
    json.dump(x_train.tolist(), f)

with open(f'{DIR_DATA_PATH}/y_train.json', 'w', encoding='utf-8') as f:
    json.dump(y_train.to_dict(orient='records'), f, ensure_ascii=False)